This is the final project "sandbox" for CSC6621 - Arsalon's Version


In [6]:
# imports 
import os
import subprocess
import shutil
import py7zr

# not required if using the Docker Image 
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator # type: ignore
import matplotlib.pyplot as plt
import numpy as np

In [17]:
from pyunpack import Archive

# Define the path to the directory containing the split archive
archive_directory = 'data/img_celeba.7z'

# Specify the first file of the split archive
first_part_path = os.path.join(archive_directory, 'img_celeba.7z.001')

# Define the output directory for the extracted files
output_directory = 'data/img_celeba_extracted'

# Extract the archive
try:
    Archive(first_part_path).extractall(output_directory)
    print("Extraction successful.")
except Exception as e:
    print("Failed to extract:", e)



Extraction successful.


An error occurred during extraction: invalid header data


In [ ]:
# Create a Train-Test Split

train_dir = os.path.join(output_folder, 'train')
test_dir = os.path.join(output_folder, 'test')


ImageDataGenerator: 

-From Tensorflow helps performing operations such as rescaling, rotating, shifting, shearing, and flipping. 
-Reads images in large numbers from a directory and automatically feed them to the model in batches

In [ ]:
# Create data generators
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)  # Use 20% of training data for validation
test_datagen = ImageDataGenerator(rescale=1./255) # rescale from (0,255) to (0,1)

# Train  Generator 
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(64, 64),  # Depending on your CNN input layer
    batch_size=32,
    class_mode='categorical',  
    subset='training'  # Set as training data
)

# Test Generator 
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(64, 64),
    batch_size=32,
    class_mode='binary'
)